# Advent of Code 2020 - Day 17

This took much longer than it should have due to the misleading example.

In [1]:
data = []
with open("inputs_day_17.txt", "r") as f:
  for line in f:
    data.append(line.strip())
data

['.#.', '..#', '###']

In [2]:
data_slice = [[d for d in row] for row in data]
dimension_size = len(data_slice)

# Finding Neighbors of a Cube

Give the coordinates denoting a cube location $(i, j, k, ...)$, we want to find the coordinates of all neigboring location, where a neighbor is defined to be the cooridnates where any coordinate differ from the original by at most $1$. So, the possibilities for the first position will be $i - 1, i, i +1$. It is easy to see that there are:

$$
3 ^ {\text{Number of Cooridnates}} - 1
$$

The neighbors can be found with recursion.

In [3]:
# Helper (src: https://code.activestate.com/recipes/579098-pick-all-combinations-of-items-in-buckets/)

def of_bucket(lst, depth = 0) :
	""" return all combinations of items in buckets """
	for item in lst[0] :
		if len(lst) > 1 :
			for result in of_bucket(lst[1:], depth + 1) :
				yield [item, ] + result
		else :
			yield [item,]

In [4]:
# Find neighbors of a cube
# Give a cube location (i, j, k, ...), it returns a list of all locations 
# where any of their coordinates differ by at most 1.
def find_cube_neighbors(cube):

  possibilites = [[coord - 1, coord, coord + 1] for coord in cube]
  neighbor_locations = []
  for n, combination in enumerate(of_bucket(possibilites)):
    if(tuple(combination) != cube):
      neighbor_locations.append(tuple(combination))

  return neighbor_locations

cube = (0, 1, 2, 3)
neighbors = find_cube_neighbors(cube)
print(neighbors)
print(len(neighbors))
#neighbor_locations

[(-1, 0, 1, 2), (-1, 0, 1, 3), (-1, 0, 1, 4), (-1, 0, 2, 2), (-1, 0, 2, 3), (-1, 0, 2, 4), (-1, 0, 3, 2), (-1, 0, 3, 3), (-1, 0, 3, 4), (-1, 1, 1, 2), (-1, 1, 1, 3), (-1, 1, 1, 4), (-1, 1, 2, 2), (-1, 1, 2, 3), (-1, 1, 2, 4), (-1, 1, 3, 2), (-1, 1, 3, 3), (-1, 1, 3, 4), (-1, 2, 1, 2), (-1, 2, 1, 3), (-1, 2, 1, 4), (-1, 2, 2, 2), (-1, 2, 2, 3), (-1, 2, 2, 4), (-1, 2, 3, 2), (-1, 2, 3, 3), (-1, 2, 3, 4), (0, 0, 1, 2), (0, 0, 1, 3), (0, 0, 1, 4), (0, 0, 2, 2), (0, 0, 2, 3), (0, 0, 2, 4), (0, 0, 3, 2), (0, 0, 3, 3), (0, 0, 3, 4), (0, 1, 1, 2), (0, 1, 1, 3), (0, 1, 1, 4), (0, 1, 2, 2), (0, 1, 2, 4), (0, 1, 3, 2), (0, 1, 3, 3), (0, 1, 3, 4), (0, 2, 1, 2), (0, 2, 1, 3), (0, 2, 1, 4), (0, 2, 2, 2), (0, 2, 2, 3), (0, 2, 2, 4), (0, 2, 3, 2), (0, 2, 3, 3), (0, 2, 3, 4), (1, 0, 1, 2), (1, 0, 1, 3), (1, 0, 1, 4), (1, 0, 2, 2), (1, 0, 2, 3), (1, 0, 2, 4), (1, 0, 3, 2), (1, 0, 3, 3), (1, 0, 3, 4), (1, 1, 1, 2), (1, 1, 1, 3), (1, 1, 1, 4), (1, 1, 2, 2), (1, 1, 2, 3), (1, 1, 2, 4), (1, 1, 3, 2), (1, 1,

In [5]:
def find_neighbor_locations_in_common(cube_locations):
  #print('find_neighbor_locations_in_common\n', cube_locations)

  # Grab neighbors for all cubes
  neighbor_locations = [] # will be a list of lists
  for cube_location in cube_locations:
    cube_neighbors = find_cube_neighbors(cube_location)
    neighbor_locations.append(cube_neighbors)

  # Find what they have in common
  common = list(set(neighbor_locations[0]).intersection(*neighbor_locations[:1]))
  #print('found in common\n', common)
  return common


In [6]:
# Given a list of cubes, return a list of active ones
def active_cube_locations_in_list(cube_locations):
  active_cubes_in_list = []
  for cube in cube_locations:
    if cube in active_cubes:
      active_cubes_in_list.append(cube)
  return active_cubes_in_list

# Given a list of cubes, return a list of inactive ones
def inactive_cube_locations_in_list(cube_locations):
  in_active_cubes_in_list = []
  for cube in cube_locations:
    if not cube in active_cubes:
      in_active_cubes_in_list.append(cube)
  return in_active_cubes_in_list

In [7]:
from collections import Counter

# Given a list of active cubes, find cube locations to acivtate
# based on the rule that inactive cube must have exactly 3 active neighbors
def find_inactive_cubes_to_activate(active_cubes):

  inactive_neighbors_of_active_cubes = []
  for active_cube in active_cubes:
    neighbors_of_active_cube = find_cube_neighbors(active_cube)
    inactive_neighbors_of_active_cubes += inactive_cube_locations_in_list(neighbors_of_active_cube)

  counts = Counter(inactive_neighbors_of_active_cubes)
  cubes_to_activate = [key for key in counts if counts[key] == 3]
  return cubes_to_activate



In [8]:
# Given an active cube location, decide whether to keep activated or deactiavte
def active_cube_activate_decision(active_cube):
  neighbors = find_cube_neighbors(active_cube)
  active_neighbors = active_cube_locations_in_list(neighbors)
  active_count = len(active_neighbors)
  return (active_count == 2 or active_count == 3)


In [9]:
def print_slice_from_3d(z, active_cubes):
  for j in range(-dimension_size, dimension_size * 2 + 1):
    for k in range(-dimension_size, dimension_size * 2 + 1):
      if((z, j, k) in active_cubes):
        print('#', end = '')
      else:
        print('.', end = '')
    print()

def print_slice_from_4d(z, w, active_cubes):
  for j in range(-dimension_size, dimension_size * 2 + 1):
    for k in range(-dimension_size, dimension_size * 2 + 1):
      if((z, w, j, k) in active_cubes):
        print('#', end = '')
      else:
        print('.', end = '')
    print()


## Part 1

In [10]:
import copy

active_cubes = []

# Initialize the grid
for i, row in enumerate(data_slice):
  for j, cube in enumerate(row):
    if(cube == '#'):
     active_cubes.append((0, i, j))

print_slice_from_3d(0, active_cubes)

# Run 6 cycles of the rules
for i in range(6):
  #print('active_cubes', active_cubes)
  active_cubes_new = copy.copy(active_cubes)

  for active_cube in active_cubes:
    if(not active_cube_activate_decision(active_cube)): # inactive
      active_cubes_new.remove(active_cube)

  inactive_cubes_to_activate = find_inactive_cubes_to_activate(active_cubes)
  active_cubes_new += inactive_cubes_to_activate
  active_cubes_new = list(set(active_cubes_new))

  #print('active_cubes_new', active_cubes_new)
  print()
  print_slice_from_3d(0, active_cubes_new)

  active_cubes = active_cubes_new

print(len(active_cubes))

..........
..........
..........
....#.....
.....#....
...###....
..........
..........
..........
..........

..........
..........
..........
..........
...#.#....
....##....
....#.....
..........
..........
..........

..........
..........
..........
..##......
..##......
..#.......
......#...
...###....
..........
..........

..........
..........
....#.....
..........
.#........
..........
......##..
..##.#....
....#.....
..........

..........
...##.....
.....#....
##...#....
###.......
#.......#.
.#......#.
.#........
......#...
...###....

..........
..#.......
..........
#.........
....#..#..
..........
........#.
.#....#.#.
.#..####..
..####....

..........
.##.......
#.#.......
.#.#####..
..........
#....#....
.....##...
.#........
.........#
..........
112


## Part 2

In [11]:
import copy

active_cubes = []

# Initialize the grid
for i, row in enumerate(data_slice):
  for j, cube in enumerate(row):
    if(cube == '#'):
     active_cubes.append((0, 0, i, j))

print_slice_from_4d(0, 0, active_cubes)

# Run 6 cycles of the rules
for i in range(6):
  #print('active_cubes', active_cubes)
  active_cubes_new = copy.copy(active_cubes)

  for active_cube in active_cubes:
    if(not active_cube_activate_decision(active_cube)): # inactive
      active_cubes_new.remove(active_cube)

  inactive_cubes_to_activate = find_inactive_cubes_to_activate(active_cubes)
  active_cubes_new += inactive_cubes_to_activate
  active_cubes_new = list(set(active_cubes_new))

  #print('active_cubes_new', active_cubes_new)
  print()
  print_slice_from_4d(0, 0, active_cubes_new)

  active_cubes = active_cubes_new

print(len(active_cubes))

..........
..........
..........
....#.....
.....#....
...###....
..........
..........
..........
..........

..........
..........
..........
..........
...#.#....
....##....
....#.....
..........
..........
..........

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
848
